In [ ]:
def load_csv(filename):
    with open(filename, 'r', encoding='utf-8-sig') as f:
        csv_list = [row.strip().split(',') for row in f.readlines()]
    return csv_list

def printx(listx):
    for x in listx:
        print(x)

def parse_row(row):
    name = row[0].lower()
    prevElemCombos = row[1].lower().split(' / ')
    prevElemCombos = [p.lower().split('+') for p in prevElemCombos]
    if prevElemCombos == [['available from start.']]:
        prevElemCombos = [['BASE_ELEMENT']]
    prevElemCombos.sort()
    for p in prevElemCombos:
        p.sort()
    return [prevElemCombo + [name] for prevElemCombo in prevElemCombos]

def parse_rows(csv_list):
    r = []
    for row in csv_list:
        r.extend(parse_row(row))
    return r
    

In [ ]:
import pandas as pd
from tqdm import tqdm

def create_elements_df(csv_list):
    r = parse_rows(csv_list)
    df = pd.DataFrame(r, columns=['elem1','elem2', 'result'])
    base_elements = get_base_elements(csv_list)
    return df[~df.index.isin(base_elements.index)]


def get_base_elements(csv_list):
    df = pd.DataFrame(parse_rows(csv_list), columns=['elem1','elem2', 'result'])
    return df[df['elem1']=='BASE_ELEMENT']

def get_elements_list(dfs):
    r = []
    for col in list(dfs.columns):
        r.extend(dfs[col].tolist())
    return list(set(r))

In [ ]:
elemsFilename = 'elements_data.csv'
csv_list = load_csv(elemsFilename)

dfs = create_elements_df(csv_list)
base_elements = get_base_elements(csv_list)['elem2'].tolist()
all_elements = get_elements_list(dfs) + base_elements
all_elements_str = ','.join(all_elements)

combos = parse_rows(csv_list)
combos_str = ';'.join([','.join(c) for c in combos])

In [ ]:
import numpy as np

def create_elements_array(all_elements, combos):
    elements_array = np.ones((len(combos), len(combos[0]))) * -1
    for i, combo in enumerate(combos):
        elements_array[all_elements.index(combo[0]), all_elements.index(combo[1])] = all_elements.index(combo[2])
    return elements_array

